In [1]:
# pos tagging for fake news statement
## use pos-tagging to build features
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
import pandas as pd
import numpy as np

# import libraries

import util
from nlp_util import NLP_Task
import preprocessing
import importlib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
import string

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [98]:
from sklearn.metrics import accuracy_score


In [4]:
# load files
tr_file, va_file, te_file = util.load_files()

In [5]:
nlpTask = NLP_Task()

NLP_Task ready to use.


In [6]:
tr_dict = util.tsv_to_dict(tsv_file=tr_file)

In [7]:
va_dict = util.tsv_to_dict(tsv_file=va_file)

In [8]:
len(tr_dict['statement'])

10240

In [10]:
stop=set(stopwords.words('english'))
text_no_stops=[]
for elm in range(0, len(tr_dict['statement'])):
    res=' '.join([i for i in tr_dict['statement'][elm].lower().split() if i not in stop])
    text_no_stops.append(res)

## remove the punctuation
def rm_punct(sentence):
    flushed_punct = set(string.punctuation)
    res = ''.join(x for x in sentence if x not in flushed_punct)
    return res

text_no_punct=[]
for i in range(0, len(tr_dict['statement'])):
    res=rm_punct(tr_dict['statement'][i])
    text_no_punct.append(res)

## here is how to clean up non-letter symbols in statement columns
all_text = []
for i in range(0, len(tr_dict['statement'])):
    patt = re.sub('[^a-zA-Z]', ' ', tr_dict['statement'][i])
    res = ' '.join(str(patt).lower().split())
    all_text.append(res)

In [10]:
corpus = []
for statement in tr_dict['statement']:
    review = re.sub('[^a-zA-Z]',' ',statement)
    review = review.lower()
    review = review.split()
    review = ''.join(review)
    corpus.append(review)

In [9]:
importlib.reload(preprocessing)

<module 'preprocessing' from 'D:\\UIC\\Fall 2018\\Statistical NLP\\Project\\jurat-aldo-project\\cs-521-project.git\\source\\CS-521-PROJECT\\preprocessing.py'>

In [11]:
unigram_pos, bigrams_pos, trigram_pos = preprocessing.extract_POS(tr_dict['statement'])

NLP_Task ready to use.
Extracting POS Tags
Finished


In [53]:
unigram_list =list()
for pos in unigram_pos:
    for p in pos:
        if p not in unigram_list:
            unigram_list.append(p)

In [55]:
len(unigram_list[1:])

45

In [45]:
#unigram_pos_va, bigrams_pos_va, trigram_pos_va = preprocessing.extract_POS(statements=va_dict['statement'])

NLP_Task ready to use.
Extracting POS Tags
Stringify
Finished


In [12]:
text_no_punct_va=[]
for i in range(0, len(va_dict['statement'])):
    res=rm_punct(va_dict['statement'][i])
    text_no_punct_va.append(res)

In [40]:
unigram_pos_va, bigrams_pos_va, trigram_pos_va = preprocessing.extract_POS(statements=va_dict['statement'])

NLP_Task ready to use.
Extracting POS Tags
Finished


In [41]:
binary_labels_tr = np.array(preprocessing.create_labels(labels=tr_dict['label'],label_values={'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))
binary_labels_va = np.array(preprocessing.create_labels(labels=va_dict['label'],label_values={'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))

In [245]:
## use countVectorizor to build postag output as features for training ML model
## uni_vocabulary: vocabulary=['cd','jjr','vb','jjs','nnp','in','vbp','to','rb','vbg','md','jj','dt']
## bi_vocabulary: vocabulary=['NNP NNP', 'IN DT', 'JJR IN', 'CD NN', 'IN CD', 'CD NNS', 'DT JJS']
## tri_vocabulary: ['NNP NNP NNP','CD NN IN','VBZ NNP NNP','IN DT NN','IN DT JJ','NN IN DT','<s> VBZ NNP','JJR IN CD','NNP NNP VBD','DT JJ CD','NNS IN DT']

# Using Count Vectorizer

In [62]:
unigram_pos_str = [" ".join(x) for x in unigram_pos]
bigram_pos_str = [" ".join(x) for x in bigrams_pos]
trigram_pos_str = [" ".join(x) for x in trigram_pos]
## use countVectorizor to build postag output as features for training ML model
#['cd','jjr','vb','jjs','nnp','in','vbp','to','rb','vbg','md','jj','dt']
cv_uni = CountVectorizer()
#cv = CountVectorizer()
pos_uni_feats = cv_uni.fit_transform(unigram_pos_str).toarray()
#pos_big_feats = cv.fit_transform(bigram_pos_str).toarray()
#pos_trig_feats = cv.fit_transform(trigram_pos_str).toarray()

In [69]:
len(cv_uni.get_feature_names())

36

In [70]:
unigram_pos_va_str = [" ".join(x) for x in unigram_pos_va]
bigram_pos_va_str = [" ".join(x) for x in bigrams_pos_va]
trigram_pos_va_str = [" ".join(x) for x in trigram_pos_va]
## use countVectorizor to build postag output as features for training ML model
cv_va = CountVectorizer(vocabulary=cv_uni.get_feature_names())
pos_uni_feats_va = cv_va.fit_transform(unigram_pos_va_str).toarray()
#pos_big_feats_va = cv.fit_transform(bigram_pos_va_str).toarray()
#pos_trig_feats_va = cv.fit_transform(trigram_pos_va_str).toarray()

In [71]:
len(cv_va.get_feature_names())

36

# Using binarizer

In [45]:
from sklearn.preprocessing import Binarizer

transformer = Binarizer().fit(pos_uni_feats[tr_indexes]) # fit does nothing.
pos_uni_feats =  transformer.transform(pos_uni_feats[tr_indexes])# preprocessing.pos_vectors(['CD','JJR','VB','JJS','NNP','IN','VBP','TO','RB','VBG','MD','JJ','DT'],unigram_pos)
transformer_va = Binarizer().fit(pos_uni_feats_va[va_indexes]) # fit does nothing.
pos_uni_feats_va = transformer.transform(pos_uni_feats_va[va_indexes])# preprocessing.pos_vectors(['CD','JJR','VB','JJS','NNP','IN','VBP','TO','RB','VBG','MD','JJ','DT'], unigram_pos_va)

IndexError: index 8126 is out of bounds for axis 0 with size 8126

# Count Vectorizer 

In [72]:
tr_indexes = [i for i,x in enumerate(binary_labels_tr) if x!=0]
va_indexes = [i for i,x in enumerate(binary_labels_va) if x!=0]
X_train = pos_uni_feats[tr_indexes]
X_test =  pos_uni_feats_va[va_indexes]
y_train = binary_labels_tr[tr_indexes]
y_test = binary_labels_va[va_indexes]

# Using own version of pos vectors

In [56]:
tr_indexes = [i for i,x in enumerate(binary_labels_tr) if x!=0]
va_indexes = [i for i,x in enumerate(binary_labels_va) if x!=0]
#transformer = Binarizer().fit(pos_uni_feats[tr_indexes]) # fit does nothing.
pos_uni_feats = preprocessing.pos_vectors(unigram_list[1:],unigram_pos)
pos_uni_feats_va = preprocessing.pos_vectors(unigram_list[1:], unigram_pos_va)
X_train = pos_uni_feats[tr_indexes]
X_test =  pos_uni_feats_va[va_indexes]
y_train = binary_labels_tr[tr_indexes]
y_test = binary_labels_va[va_indexes]

Processing POS tags and creating vectors
Finished
Processing POS tags and creating vectors
Finished


In [116]:
m_clf = DecisionTreeClassifier(max_depth=5,criterion='entropy')
m_clf.fit(X_train, y_train)
predicted = m_clf.predict(X_test)
print(m_clf.score(X=X_test, y=y_test))
print(cross_val_score(m_clf, X_test, y_test, cv=10))

0.6283783783783784
[0.58653846 0.53846154 0.56730769 0.625      0.60576923 0.625
 0.6407767  0.59223301 0.66019417 0.66019417]


In [81]:
tr_indexes = [i for i,x in enumerate(binary_labels_tr) if x!=0]
va_indexes = [i for i,x in enumerate(binary_labels_va) if x!=0]
X_train = pos_uni_feats[tr_indexes]
X_test =  pos_uni_feats_va[va_indexes]
y_train = binary_labels_tr[tr_indexes]
y_test = binary_labels_va[va_indexes]

# Training Different Models 

In [115]:
## Need to Provide Xtrain, Ytrain, Xtest, Ytest
Xtrain = X_train
Ytrain = y_train
Xtest = X_test
Ytest = y_test
kf = KFold(n_splits=10, shuffle=True)
# ,'class_weight':{1:.9, 2:.5, 3:.01}
params = [{}, {'loss': 'log', 'penalty': 'l2', 'max_iter':1000},{},{},{}]
Models = [LogisticRegression, linear_model.SGDClassifier, MultinomialNB, RandomForestClassifier, SVC]
accuracy_list = list()
accuracy_metrics = list()
for param, Model in zip(params, Models):
    total = 0
    for train_indices, test_indices in kf.split(X_train):
        train_X = Xtrain[train_indices, :]; train_Y = Ytrain[train_indices]
        test_X = Xtrain[test_indices, :]; test_Y = Ytrain[test_indices]
        reg = Model(**param)
        reg.fit(train_X, train_Y)
        predictions = reg.predict(Xtest)
        total += accuracy_score(Ytest, predictions)
    
    accuracy = total / kf.n_splits
    reg = Model(**param)
    reg.fit(X_train, y_train)
    predictions = reg.predict(X_test)
    accuracy_list.append((Model.__name__, accuracy))
    accuracy_metrics.append((Model.__name__, classification_report(y_test, predictions)))
    
for i, value in enumerate(accuracy_list):
    print("Accuracy score of {0}: {1}".format(value[0],value[1]))
    print("accuracy metrics for {0}:\n{1}".format(accuracy_metrics[i][0], accuracy_metrics[i][1]))
    
#print("Accuracy score of {0}: {1}: {2}: {3}: {4}".format(accuracy_list[0], accuracy_list[]))
#print("accuracy metrics for {0}: {1}: {2}: {3}: {4}".format(Model.__name__, classification_report(y_test, predictions)))

Accuracy score of LogisticRegression: 0.6236486486486488
accuracy metrics for LogisticRegression:
             precision    recall  f1-score   support

         -1       0.55      0.41      0.47       420
          1       0.66      0.78      0.71       616

avg / total       0.62      0.63      0.61      1036

Accuracy score of SGDClassifier: 0.6214285714285716
accuracy metrics for SGDClassifier:
             precision    recall  f1-score   support

         -1       0.58      0.33      0.42       420
          1       0.65      0.83      0.73       616

avg / total       0.62      0.63      0.60      1036

Accuracy score of MultinomialNB: 0.6219111969111969
accuracy metrics for MultinomialNB:
             precision    recall  f1-score   support

         -1       0.54      0.53      0.53       420
          1       0.68      0.69      0.68       616

avg / total       0.62      0.62      0.62      1036

Accuracy score of RandomForestClassifier: 0.5615830115830116
accuracy metrics for